In [ ]:
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting


In [ ]:
print(os.listdir('../input'))

In [ ]:
data_dir = "../input/depthgestrecog/depthGestRecog/"   

In [ ]:
label_dict = {}
# get the class num
for label in os.listdir(data_dir):
   for sub_dir in os.listdir(os.path.join(data_dir,label)):
       class_num = int((os.listdir(os.path.join(data_dir,label,sub_dir)))[:][0][5:7])
   label_dict[class_num] = label

In [ ]:
x = []
y = []
for root, _, files in os.walk(data_dir):
    for i in range(len(files)):
        x.append((plt.imread((os.path.join(root,files[i])))))
        y.append((int(files[i][5:7])))

x = np.asarray(x, dtype=np.float32)
y = np.asarray(y)

In [ ]:
y

In [ ]:
y_ohev = []

for j in y:
    ohev = np.zeros(11)  # 11 classes ( replace by no of classes)
    ohev[j - 1] = 1
    y_ohev.append(ohev)

In [ ]:
y_ohev

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y_ohev,test_size = 0.2)


In [ ]:
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 3

for i in range(1, columns * rows + 1):
    gest = ""
    ind = np.random.randint(len(x_train))
    fig.add_subplot(rows, columns, i)
    for key, value in label_dict.items():
        if value == (np.argmax(y_train[ind]) + 1):
            gest = key
    plt.title(gest)
    plt.imshow(x_train[ind])

plt.show()

In [ ]:
x_train = np.array(x_train)
x_train[x_train > 0] = 1
x_test = np.array(x_test)
x_test[x_test > 0] = 1

y_test = np.array(y_test)
y_train = np.array(y_train)


In [ ]:
from keras import backend as K
num_classes = 11
img_rows = 100
img_cols = 100
# input shape (100, 100, 1)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Now it's time to build our network. We'll use keras.

In [ ]:
from keras import layers
from keras import models

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=input_shape)) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

Finally, we fit the model.

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_test, y_test))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# plot for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=0)
print("Accuracy:" + str(acc))